In [1]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd


In [2]:
def extract_webpage_info(url):
    response = requests.get(url)

    soup = BeautifulSoup(response.content, "html.parser")

    title = soup.title.string if soup.title else "No Title Found"

    links = [a["href"] for a in soup.find_all("a", href=True)]

    headers = {}
    for i in range(1, 7):
        headers[f"h{i}"] = [h.get_text() for h in soup.find_all(f"h{i}")]

    return {"Title": title, "Links": links, "Headers": headers}

In [6]:
url = input("Enter the URL to scrape: ")
webpage_info = extract_webpage_info(url)

with open(
    "webpage_data.json", "w", encoding="utf-8"
) as file:
    json.dump(webpage_info, file, ensure_ascii=False, indent=4)

In [7]:
with open(
    "webpage_data.json", "r", encoding="utf-8"
) as file:
    data = json.load(file)

if isinstance(data, dict):
    web_df = pd.json_normalize(data)
else:
    web_df = pd.DataFrame(data)

In [34]:
page_links =  web_df['Links'][0]

link_series = pd.Series(page_links).astype(str)

In [35]:
link_series = link_series.drop(link_series[link_series.str.contains('javascript')].index)
link_series

0                      https://www.tgju.org/applications
1         https://cafebazaar.ir/app/com.vandaw.tgju?l=en
2              https://www.tgju.org/applications/#ios-dl
3                      https://www.tgju.org/applications
4                                                      .
                             ...                        
593                              https://m.me/TgjuSocial
594    https://slack.com/oauth/v2/authorize?client_id...
595                           https://wa.me/989981911511
597                                                    #
601                                                    #
Length: 597, dtype: object

In [36]:
link_series.str.contains('https')

0       True
1       True
2       True
3       True
4      False
       ...  
593     True
594     True
595     True
597    False
601    False
Length: 597, dtype: bool

In [37]:
clean_links = link_series[link_series.str.contains('https')]


In [38]:
clean_links.dropna(inplace=True)

In [42]:
clean_links = clean_links.reindex()

In [48]:
len(clean_links)

154

In [23]:
web_df

,Title,Links,Headers.h1,Headers.h2,Headers.h3,Headers.h4,Headers.h5,Headers.h6
0,سکه امامی - شبکه اطلاع رسانی طلا و ارز,"[https://www.tgju.org/applications, https://ca...",[],"[اپلیکیشن, تاریخچه سکه امامی, آیا مایلید از ن...",[],[],"[دلار, یورو, پوند, انس طلا, مثقال, طلای 18, طل...","[قیمت طلا, قیمت سکه, نرخ ارز, قیمت دلار, ارزه..."


In [24]:
web_df['Headers.h2'][0]

['اپلیکیشن', 'تاریخچه سکه امامی', ' آیا مایلید از نوسانات بازار آگاه شوید؟ ']

In [25]:
header_2_series = pd.Series(web_df['Headers.h2'][0])
header_2_series

0                                    اپلیکیشن
1                           تاریخچه سکه امامی
2     آیا مایلید از نوسانات بازار آگاه شوید؟ 
dtype: object

In [27]:
header_5_series = pd.Series(web_df['Headers.h5'][0])
header_5_series

0          دلار
1          یورو
2          پوند
3       انس طلا
4         مثقال
5       طلای 18
6       طلای 24
7         آبشده
8     سکه امامی
9           نیم
10          ربع
11         گرمی
12     نفت برنت
13     بیت کوین
14       اتریوم
15          تتر
dtype: object

In [28]:
header_6_series = pd.Series(web_df['Headers.h6'][0])
header_6_series

0          قیمت طلا
1          قیمت سکه
2           نرخ ارز
3         قیمت دلار
4    ارزهای دیجیتال
5    بورس و فرابورس
6       بازار جهانی
7       بازار داخلی
dtype: object

In [44]:
title_series = pd.Series(web_df['Title'])

title_series

0    سکه امامی - شبکه اطلاع رسانی طلا و ارز
Name: Title, dtype: object

In [49]:
clean_links

0                      https://www.tgju.org/applications
1         https://cafebazaar.ir/app/com.vandaw.tgju?l=en
2              https://www.tgju.org/applications/#ios-dl
3                      https://www.tgju.org/applications
6      https://myportal.errantetrading.net/marketing/...
                             ...                        
591                                https://t.me/tgjunews
592                         https://t.me/TGJU_Supportbot
593                              https://m.me/TgjuSocial
594    https://slack.com/oauth/v2/authorize?client_id...
595                           https://wa.me/989981911511
Length: 154, dtype: object

In [51]:
df = pd.concat([title_series, clean_links, header_2_series, header_5_series, header_6_series], axis=1)
df.columns = ['Title', 'Links', 'Header_2', 'Header_5', 'Header_6']
df

,Title,Links,Header_2,Header_5,Header_6
0,سکه امامی - شبکه اطلاع رسانی طلا و ارز,https://www.tgju.org/applications,اپلیکیشن,دلار,قیمت طلا
1,NaN,https://cafebazaar.ir/app/com.vandaw.tgju?l=en,تاریخچه سکه امامی,یورو,قیمت سکه
2,NaN,https://www.tgju.org/applications/#ios-dl,آیا مایلید از نوسانات بازار آگاه شوید؟,پوند,نرخ ارز
3,NaN,https://www.tgju.org/applications,NaN,انس طلا,قیمت دلار
6,NaN,https://myportal.errantetrading.net/marketing/...,NaN,طلای 24,بازار جهانی
...,...,...,...,...,...
11,NaN,NaN,NaN,گرمی,NaN
12,NaN,NaN,NaN,نفت برنت,NaN
13,NaN,NaN,NaN,بیت کوین,NaN
14,NaN,NaN,NaN,اتریوم,NaN


In [52]:
json_str = df.to_json(orient='records', force_ascii=False)
with open("combined_series.json", "w", encoding="utf-8") as file:
    file.write(json_str)